In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# データの例を作成
data = {
    'date': ['31-Mar-19', '31-Mar-19', '31-Mar-19'],
    'time': ['12:36:52', '12:36:53', '12:36:54'],
    'fridge_temperature': [13.1, 8.65, 2],
    'temp_condition': ['high', 'high', 'low'],
    'label': [0, 0, 1],
    'type': ['normal', 'normal', 'ddos']
}

df = pd.DataFrame(data)

# temp_conditionのワンホットエンコーディング
encoder = OneHotEncoder(sparse=False)
encoded_conditions = encoder.fit_transform(df[['temp_condition']])
condition_df = pd.DataFrame(encoded_conditions, columns=encoder.get_feature_names_out())

# データフレームを統合
df = df.join(condition_df)
df.drop(['date', 'time', 'temp_condition', 'type'], axis=1, inplace=True)

# 特徴量とラベルを分割
X = df.drop('label', axis=1)
y = df['label']

# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 特徴量のスケーリング
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


/home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# モデルの構築
model = Sequential([
    LSTM(50, input_shape=(X_train_scaled.shape[1], 1)),  # 入力形状に注意
    Dense(1, activation='sigmoid')
])

# モデルのコンパイル
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


2024-07-03 15:53:17.462471: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 15:53:17.835353: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 15:53:18.426026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-03 15:53:19.062104: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

In [6]:
# LSTMに合うようにデータの形状を変更
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# モデルのトレーニング
history = model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))


Epoch 1/10


2024-07-03 15:53:30.486418: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.6849 - val_accuracy: 1.0000 - val_loss: 0.6591
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 0.6817 - val_accuracy: 1.0000 - val_loss: 0.6523
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.6784 - val_accuracy: 1.0000 - val_loss: 0.6456
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.6750 - val_accuracy: 1.0000 - val_loss: 0.6389
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.6716 - val_accuracy: 1.0000 - val_loss: 0.6322
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.6682 - val_accuracy: 1.0000 - val_loss: 0.6254
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.6647 - val_accuracy: 1.0000 - val_loss: 0.6186
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.6612 - val_accuracy: 1.0000 - val_loss: 0.6117
Epoch 9/10
1/

In [7]:
# モデルの評価
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.5974
Test Accuracy: 100.00%
